This notebook demonstrates strain mapping using both of the methods implemented in pyXem.

## Requirements

pyXem beyond commit d1dc645 - which came in Jan 28th 2019

# Contents

1. <a href='#loa'> Simulate Strain Test Data</a>
2. <a href='#str'> Strain Mapping as Image Affine Transform</a>
3. <a href='#str'> Strain Mapping as Image Affine Transform</a>

Import pyXem and other required libraries

In [23]:
%matplotlib tk
import pyxem as pxm
import numpy as np
import hyperspy.api as hs
import diffpy.structure
from matplotlib import pyplot as plt
from pyxem.generators.indexation_generator import IndexationGenerator


## <a id='loa'></a> 4. Simulate Strain Test Data

If you have data ready to go you can skip this section. We start by defining a crystal structure

In [113]:
#An atomic structure represented using diffpy
latt = diffpy.structure.lattice.Lattice(3,3,5,90,90,90)
atom = diffpy.structure.atom.Atom(atype='Ni',xyz=[0,0,0],lattice=latt)
hexagonal_structure = diffpy.structure.Structure(atoms=[atom],lattice=latt)

We now perform our (virtual) diffraction experiment to create a single pattern

In [114]:
ediff = pxm.DiffractionGenerator(300., 0.025)
diffraction = ediff.calculate_ed_data(hexagonal_structure, reciprocal_radius=5.)
pattern = diffraction.as_signal(128,0.05,1).data

In [115]:
plt.imshow(pattern)

and a spare, so we can see the effect of a real space change

In [140]:
latt = diffpy.structure.lattice.Lattice(3+0.12,3+0.12,3,90,90,90)
atom = diffpy.structure.atom.Atom(atype='Ni',xyz=[0,0,0],lattice=latt)
hexagonal_structure_d = diffpy.structure.Structure(atoms=[atom],lattice=latt)
diffractiond = ediff.calculate_ed_data(hexagonal_structure_d, reciprocal_radius=5.)
patternd = diffractiond.as_signal(128,0.05,1).data

In [143]:
0.12/3

0.04

Four copies of this pattern are made, and strained by different amounts (note we conduct the deformation in reciprocal space here to keep the code volume down)

In [144]:
dp = pxm.ElectronDiffraction((np.asarray([[pattern,patternd],[pattern,pattern]])))

x_l = []
for x in [0, 0, -0.01, 0.02]:
    x_s = np.eye(3)
    x_s[0,0] += x
    x_l.append(x_s)

angles = hs.signals.Signal2D(np.asarray(x_l).reshape(2,2,3,3))
dp = dp.apply_affine_transformation(D=angles,order=5,inplace=False)
dp.set_diffraction_calibration(1)

In [145]:
%matplotlib tk
dp.plot(cmap='magma')

And this data is ready to be worked on!

## <a id='loa'></a> 5. Perform Strain Mapping with an affine transform method

Create a model to the data which comprises a distorted version of a reference (unstrained) diffraction pattern at each probe position.

In [146]:
from pyxem.components.scalable_reference_pattern import ScalableReferencePattern

In [147]:
m = dp.create_model()
ref = ScalableReferencePattern(dp.inav[0,0])
m.append(ref)

Print the affine transform values associated with the distorted diffraction pattern before fitting.

In [148]:
m.print_current_values()

Components	Parameter	Value
ScalableReferencePattern
		d11	1
		d12	0
		d21	0
		d22	1
		t1	0
		t2	0


Perform fitting

In [149]:
m.multifit()

Construct the displacement graident tensor at each pixel from the fitting results

In [164]:
disp_grad = ref.construct_displacement_gradient()

In [167]:
disp_grad.inav[1,1].isig[0,0].data

array([1.02001305])

Perform (right) polar decomposition on the displacement gradient tensor to get rotation matrix, R, and strain matrix, U, at each pixel.

In [151]:
R, U = disp_grad.polar_decomposition()

Get a strain map and plot it.

In [152]:
strain_map = disp_grad.get_strain_maps()
strain_map.plot()

In [126]:
strain_map.data

array([[[ 0.00000000e+00, -8.18133668e-04],
        [ 1.00932736e-02, -2.00130237e-02]],

       [[ 0.00000000e+00, -8.18133047e-04],
        [ 8.54398595e-07, -9.13212320e-06]],

       [[ 0.00000000e+00,  4.91334948e-07],
        [-6.05508190e-08, -7.05014860e-08]],

       [[-0.00000000e+00,  2.01503716e-07],
        [ 9.58144949e-08,  3.52728154e-08]]])

Note: This method is very succesful with the designed data as they both rely on affine transform algorithms

## <a id='loa'></a> 6. Perform Strain Mapping with a spot tracking method

In [153]:
from pyxem.generators.subpixelrefinement_generator import SubpixelrefinementGenerator
from pyxem.signals.tensor_field import *
from pyxem.generators.displacement_gradient_tensor_generator import *

We start by finding the two peaks, this can be done with other peak finding methods, or by inspection (as here), although we must calibrate the signal for this

In [154]:
dp.plot()

#help(get_DisplacementGradientMap) #this is sometimes useful to read for conventions

x_peak = [24,0]
y_peak = [0,-42]

We need to find these peaks (in each pattern) at subpixel precision

In [155]:
spg = SubpixelrefinementGenerator(dp,np.asarray([x_peak,y_peak]))
Vs = spg.center_of_mass_method(6)

In [156]:
Vs

array([[[[ 23.4633089 ,   0.        ],
         [  0.        , -42.        ]],

        [[ 23.30137082,   0.        ],
         [  0.        , -41.81588602]]],


       [[[ 23.41838832,   0.        ],
         [  0.        , -42.        ]],

        [[ 23.55067089,   0.        ],
         [  0.        , -42.        ]]]])

In [169]:
23.55 / 23.46

1.0038363171355498

And with these objects (deformed and undeformed peaks) we are off to the races

In [ ]:
D = get_DisplacementGradientMap(hs.signals.Signal2D(Vs), Vs[0,0])

In [168]:
D.inav[1,1].isig[0,0].data

array([1.00372334])

In [157]:
strain_map = D.get_strain_maps()

In [160]:
D.data

array([[[[1.        , 0.        , 0.        ],
         [0.        , 1.        , 0.        ],
         [0.        , 0.        , 1.        ]],

        [[0.99309824, 0.        , 0.        ],
         [0.        , 0.99561633, 0.        ],
         [0.        , 0.        , 1.        ]]],


       [[[0.9980855 , 0.        , 0.        ],
         [0.        , 1.        , 0.        ],
         [0.        , 0.        , 1.        ]],

        [[1.00372334, 0.        , 0.        ],
         [0.        , 1.        , 0.        ],
         [0.        , 0.        , 1.        ]]]])

In [158]:
strain_map.plot(cmap='seismic')

In [159]:
strain_map.data

array([[[ 0.        ,  0.00690176],
        [ 0.0019145 , -0.00372334]],

       [[ 0.        ,  0.00438367],
        [ 0.        ,  0.        ]],

       [[ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[-0.        , -0.        ],
        [-0.        , -0.        ]]])

In [44]:
raise KeyboardInterrupt

KeyboardInterrupt: 

In [ ]:
#plt.imshow(dp.inav[0,1].data[60:68,60+24:68+24])

In [ ]:
from pyxem.utils.subpixel_refinements_utils import get_experimental_square

In [ ]:
z = dp.inav[0,1].data

In [ ]:
plt.imshow(get_experimental_square(z,vector=[64,64],square_size=16))

In [ ]:
z_r = get_experimental_square(z,vector=[64,64],square_size=16)
z_r[0,:] = 0
z_r[:,0] = 0

In [ ]:
def _center_of_mass_hs(z):
    from scipy.ndimage.measurements import center_of_mass
    t = center_of_mass(z)
    x = t[1]
    y = t[0]
    return (x,y)

In [ ]:
_center_of_mass_hs(z_r)

In [ ]:
#get_experimental_square(z,vector=[4,4],square_size=3)

In [ ]:
#cy = 4
#cx = 4
#r = 1

#_z = z[cy - r:cy + r + 1, cx - r:cx + r + 1]

In [ ]:
#_z

In [ ]:
#z[4,4]

In [ ]:
#import scipy
#scipy.ndimage.measurements.center_of_mass(_z)